# Python Code for Blog Post 04/Aug/2023

The present notebook contains the python code used for the Blog Post written on 04/Aug/2023
about LLMs producing JSON to interface with applications.

# Environment Setup

In [1]:
%%capture
%pip install openai
%pip install requests
%pip install beautifulsoup4

## Scraping Blog Post

In [2]:
import requests
from bs4 import BeautifulSoup

blog_url = "https://medium.com/@hudsonmendes/robo-3t-dangerous-default-value-for-create-index-in-background-7cb2f6bc9a78"
blog_page = requests.get(blog_url)
blog_soup = BeautifulSoup(blog_page.content, "html.parser")
blog_text = blog_soup.get_text("\n").strip()
blog_text[0:1000] + " (...)"

'Robo 3T, Dangerous default value for "Create Index in Background" | by ⚡️Hudson Ⓜ️endes | Medium\nRobo 3T, Dangerous default value for "Create Index in Background"\n⚡️Hudson Ⓜ️endes\n·\nFollow\n3 min read\n·\nApr 16, 2020\n--\nListen\nShare\nIndexes in mongo must be created in Background. Not doing it is dangerous enough to bring your entire MongoDB cluster down, and beyond repair.\nProblem\nCreate and index, followed by any modification to the index in a large collection with the \nAdvanced > Create Index in Background\n option set to \nfalse\n.\nThe Cluster propagates those actions and becomes unresponsive!\nGithub Issue\n"Create index in background" checked by default · Issue #1656 · Studio3T/robomongo\nDismiss GitHub is home to over 40 million developers working together to host and review code, manage projects, and…\ngithub.com\nHistory\nNot once, but twice, I have created and shortly after modified an index in different collections that were both large.\nIn the first occurrence,

## Prompting LLM (GPT-4)

In [3]:
import os
import openai
with open("/Users/hudsonmendes/.openai/key", "r", encoding="utf-8") as fh:
    openai.api_key = fh.read().strip()

In [4]:
from string import Template
prompt_template = Template("""
Extract the following attributes from the blog post: 
- Main Topic of the Post
- Overall, does the post express a positive or negative view?
- If available, what is the call to action?

The blog post is delimited with triple hashes. \
Format your response as a JSON object with \
"main_topic", "sentiment_polarity" and "call_to_action"as the keys.
If the information isn't present, use `null` \
as the value.
Make your response as short as possible.
Format the "sentiment_polarity" value as a range between -1 for negative and 1 for positive.

Blog Post: ###${text}###
""")

In [5]:
import json
from IPython.display import display, JSON

completion = openai.ChatCompletion.create(
    model="gpt-4-0613",
    temperature=0,
    messages=[
        {"role": "system", "content": "You are an information extraction system."},
        {"role": "user", "content": prompt_template.substitute(text=blog_text)}
    ]
)

completion

<OpenAIObject chat.completion id=chatcmpl-7jkVZVwmsIhAybABFHVNM9NYSbIrw at 0x112af4540> JSON: {
  "id": "chatcmpl-7jkVZVwmsIhAybABFHVNM9NYSbIrw",
  "object": "chat.completion",
  "created": 1691137893,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\"main_topic\": \"The dangers of not creating indexes in the background in MongoDB using Robo 3T\", \"sentiment_polarity\": -0.5, \"call_to_action\": \"Consider creating indexes in background unless you explicitly need to do otherwise and follow the author on Twitter for more updates\"}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1720,
    "completion_tokens": 62,
    "total_tokens": 1782
  }
}

## Parsing Results

In [6]:
display(JSON(json.loads(completion.choices[0].message.content)))

<IPython.core.display.JSON object>